In [1]:
import tomotopy as tp
from sklearn import preprocessing
import numpy as np
import csv
import nltk
import string
from sklearn import metrics
import sklearn.metrics.pairwise as smp
from sklearn_extra.cluster import KMedoids
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from dateutil import parser
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import sklearn.cluster as sc
nltk.download('stopwords')
s=set(stopwords.words('english'))
import pandas as pd
import skfuzzy as fuzz

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lorenchamplin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lorenchamplin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lorenchamplin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lorenchamplin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def lowercase(input):
  """
  Returns lowercase text
  """
  return input.lower()

def remove_punctuation(input):
  """
  Returns text without punctuation
  """
  return input.translate(str.maketrans('','', string.punctuation+'–’”“—·'))

def remove_whitespaces(input):
  """
  Returns text without extra whitespaces
  """
  return " ".join(input.split())
  
def remove_html_tags(input):
  """
  Returns text without HTML tags
  """
  soup = BeautifulSoup(input, "html.parser")
  stripped_input = soup.get_text(separator=" ")
  return stripped_input

def tokenize(input):
  """
  Returns tokenized version of text
  """
  return word_tokenize(input)

def remove_stop_words(input):
  """
  Returns text without stop words
  """
  input = word_tokenize(input)
  return [word for word in input if word not in stopwords.words('english') or word == "no" or word == "not"]

def lemmatize(input):
  """
  Lemmatizes input using NLTK's WordNetLemmatizer
  """
  lemmatizer=WordNetLemmatizer()
  input_str=word_tokenize(input)
  new_words = []
  for word in input_str:
    new_words.append(lemmatizer.lemmatize(word))
  return ' '.join(new_words)


def nlp_pipeline(input):
  """
  Function that calls all other functions together to perform NLP on a given text
  """
  return lemmatize(' '.join(remove_stop_words(remove_whitespaces(remove_punctuation(remove_html_tags(lowercase(input)))))))

In [3]:
mdl = tp.LDAModel(tw=tp.TermWeight.ONE,k=6,seed=2021)
data = []
sentences = []
truth_labels = []
with open("../comments_2.csv", newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        truth_labels.append(row[1])
        sentences.append(row[0])
        line = nlp_pipeline(row[0]).split()
        if line:
            mdl.add_doc(nlp_pipeline(row[0]).split())
            data.append(nlp_pipeline(row[0]).split())

In [4]:
for i in range(0, 500, 10):
    mdl.train(10)

mdl.summary()

<Basic Info>
| LDAModel (current version: 0.12.2)
| 694 docs, 21405 words
| Total Vocabs: 3117, Used Vocabs: 3117
| Entropy of words: 6.48915
| Entropy of term-weighted words: 6.48915
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 500, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -6.96409
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 6 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 2021 (random seed)
| trained in version 0.12.2
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [0.2959824  0.20503

In [5]:
samples = []
for i in data:
    samples.append(mdl.make_doc(i))

In [6]:
X = mdl.infer(samples)

In [7]:
X = np.array(X[0])

In [8]:
def cluster_and_evaluate(texts,K,X,truth_labels):
    clustering = KMedoids(n_clusters=K,method='pam',metric='cosine',init='k-medoids++')
    clustering.fit(X)
    print("\n")
    print("=== Unsupervised Metric ===\n")
    print("== Silhouette Coefficient [0,1*]==")
    print(metrics.silhouette_score(X, clustering.labels_, metric='cosine'),"\n")
    print("\n")
    print("=== Supervised Metric ===\n")
    print("== V-measure [0,1*] ==")
    print(metrics.v_measure_score(truth_labels, clustering.labels_),"\n")
    print("\n")
    print("=== Clusters Exemplars ===\n")
    for i,t in enumerate(clustering.medoid_indices_):
        print(i,": ",texts[t],"\n")

In [9]:
cluster_and_evaluate(sentences,90,X,truth_labels)

[456 361 551 467  18  27  89  54  25   6 260 232 228 439  36  91 393 346
 595 296 488 667 323 353 273 401 189 652 687 287 500 312 332 621 521 533
 236 482 578 252 610  64 473 567  63 131 536 251 225 253 460 379 294 315
 180 123 321 433 192 597 583  83 387  40 645 425 384 381  92 661 681 124
 214 175 300 527  39 201 313 334 465 444 543 247 637 628 118 539 574 464]


=== Unsupervised Metric ===

== Silhouette Coefficient [0,1*]==
0.5081756 



=== Supervised Metric ===

== V-measure [0,1*] ==
0.35618771273127353 



=== Clusters Exemplars ===

0 :  Arizonan’s want you to hire an unbiased, fair, and independent company to preform our
Az redistricting. We would like for you to hire National Demographic Corp.  Thank you 

1 :  Dear Commissioners,
Please hire only the most experienced mapping agency. There is no room for steep
learning curves, or starting from scratch. This redistricting project deserves only the most
qualified and proven outfit, especially when you account for the shortened